In [26]:
import os
import logging

import psycopg2
import psycopg2.extensions
from pymongo import MongoClient
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, Float, MetaData, String
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from subprocess import call


logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

call("bash load_data.sh", shell=True)

127

In [27]:

logger.info("Создаём подключёние к Postgres")
params = {
    "host": 'postgres_host',
    "port": '5432',
    "user": 'postgres'
}
conn = psycopg2.connect(**params)

# дополнительные настройки
psycopg2.extensions.register_type(
    psycopg2.extensions.UNICODE,
    conn
)
conn.set_isolation_level(
    psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT
)
cursor = conn.cursor()

2018-08-27 16:25:30,194 : INFO : Создаём подключёние к Postgres


In [28]:

# ВАШ КОД ЗДЕСЬ
# -------------
# таблица movies_top
# movieId (id фильма), ratings_num(число рейтингов), ratings_avg (средний рейтинг фильма)

# "SELECT movieId, ratings_num, rating_avg INTO movies_top"
#drop table movies_top;
sql_str = """
SELECT movieid , count(rating) ratings_num, avg(rating) rating_avg
INTO movies_top
from public.ratings 
group by movieid; 
"""
# -------------

cursor.execute(sql_str)
conn.commit()

sql_str = ""

ProgrammingError: relation "movies_top" already exists


In [29]:
# Проверка - выгружаем данные

sql_str = "select  movieid , ratings_num, rating_avg from movies_top where rating_avg >3 limit 10;"


cursor.execute(sql_str)
logger.info(
    "Выгружаем данные из таблицы movies_top: (movieId, ratings_num, ratings_avg)\n{}".format(
        [i for i in cursor.fetchall()])
)


2018-08-27 16:25:38,312 : INFO : Выгружаем данные из таблицы movies_top: (movieId, ratings_num, ratings_avg)
[(6114, 1, 3.5), (273, 27, 3.12962962962963), (128606, 1, 4.5), (4326, 11, 4.09090909090909), (2520, 15, 3.36666666666667), (25886, 1, 4.0), (4321, 28, 3.46428571428571), (5230, 1, 4.0), (176, 18, 3.77777777777778), (4993, 200, 4.1825)]


In [30]:

# Задание по SQLAlchemy
# --------------------------------------------------------------
Base = declarative_base()


class MoviesTop(Base):
    __tablename__ = 'movies_top'

    movieid = Column(Integer, primary_key=True)
    ratings_num = Column(Integer)
    rating_avg = Column(Float)

    def __repr__(self):
        return "<User(movieid='%s', ratings_num='%s', rating_avg='%s')>" % (self.movieid, self.ratings_num, self.rating_avg)



In [31]:
# Создаём сессию

engine = create_engine('postgresql://postgres:@{}:{}'.format('postgres_host', '5432'))
Session = sessionmaker(bind=engine)
session = Session()

In [32]:
# --------------------------------------------------------------
# Ваш код здесь
# выберите контент у которого больше 15 оценок (используйте filter)
# и средний рейтинг больше 3.5 (filter ещё раз)
# отсортированный по среднему рейтингу (используйте order_by())
# id такого контента нужно сохранить в массив top_rated_content_ids


top_rated_query = session.query(MoviesTop).filter("rating_avg > 3.5").filter("ratings_num > 15")

logger.info("Выборка из top_rated_query\n{}".format([i for i in top_rated_query.limit(4)]))

top_rated_content_ids = [
    i[0] for i in top_rated_query.values(MoviesTop.movieid)
][:5]
# --------------------------------------------------------------

/opt/conda/lib/python3.6/site-packages/sqlalchemy/sql/elements.py:4267: SAWarning: Textual SQL expression 'rating_avg > 3.5' should be explicitly declared as text('rating_avg > 3.5') (this warning may be suppressed after 10 occurrences)
  {"expr": util.ellipses_string(element)})
2018-08-27 16:25:45,860 : INFO : Выборка из top_rated_query
[<User(movieid='176', ratings_num='18', rating_avg='3.77777777777778')>, <User(movieid='4993', ratings_num='200', rating_avg='4.1825')>, <User(movieid='161', ratings_num='80', rating_avg='3.83125')>, <User(movieid='8873', ratings_num='26', rating_avg='3.67307692307692')>]


In [34]:
# Задание по PyMongo

#call("bash mongoimport --host $APP_MONGO_HOST --port $APP_MONGO_PORT --db movies --collection tags --file simple_tags.json", shell=True)

mongo = MongoClient('mongo_host',27017) #
mongo.server_info() # Forces a call.

#call("bash load_json.sh", shell=True)
call("""bash "mongoimport --host $APP_MONGO_HOST --port $APP_MONGO_PORT --db movies --collection tags --file work/simple_tags.json""", shell=True) 
# Получите доступ к коллекции tags
db = mongo["movie"]
tags_collection = db['tags']
#db.stats()
db.command("dbstats")

{'db': 'movie',
 'collections': 0,
 'views': 0,
 'objects': 0,
 'avgObjSize': 0,
 'dataSize': 0,
 'storageSize': 0,
 'numExtents': 0,
 'indexes': 0,
 'indexSize': 0,
 'fileSize': 0,
 'fsUsedSize': 0,
 'fsTotalSize': 0,
 'ok': 1.0}

In [47]:
# id контента используйте для фильтрации - передайте его в модификатор $in внутри find
# в выборку должны попать теги фильмов из массива top_rated_content_ids

top_rated_content_ids = tags_collection.find().limit(3)

top_rated_content_ids_docs = [
    i for i in top_rated_content_ids
]

print("Достали документы из Mongo: {}".format(top_rated_content_ids_docs[:5]))
id_tags = [(i['id'], i['name']) for i in top_rated_content_ids_docs]


mongo_query = tags_collection.find( 
{'id': {}}
)

mongo_docs = [
    i for i in mongo_query
]

print("Достали документы из Mongo: {}".format(mongo_docs[:5]))

id_tags = [(i['id'], i['name']) for i in mongo_docs]




Достали документы из Mongo: []
Достали документы из Mongo: []


In [49]:
id_tags

[]

In [ ]:

# Задание по Pandas
# --------------------------------------------------------------
# Постройте таблицу их тегов и определите top-5 самых популярных

# формируем DataFrame
tags_df = pd.DataFrame(id_tags, columns=['movieid', 'tags'])

# --------------------------------------------------------------
# Ваш код здесь
# сгруппируйте по названию тега с помощью group_by
# для каждого тега вычислите, в каком количестве фильмов он встречается
# оставьте top-5 самых популярных тегов

top_5_tags = tags_df.head(5)

print(top_5_tags)

logger.info("Домашка выполнена!")
# --------------------------------------------------------------


In [22]:
files = os.listdir(os.curdir)

In [24]:
files

['.bash_logout',
 '.profile',
 '.bashrc',
 'data',
 'account_count.csv',
 '.local',
 'account_counter.csv',
 'Project_20180824.ipynb',
 'Project_SQL-Copy1.ipynb',
 'accounts.csv',
 'HW6_2.ipynb',
 'Project_SQL.ipynb',
 'notebook.tex',
 '.jupyter',
 '.ipython',
 'window_sums.csv',
 '.ipynb_checkpoints',
 'Untitled.ipynb',
 '.conda',
 '.cache',
 '.config',
 '.yarn',
 'work']